<a href="https://colab.research.google.com/github/lee-j-j/LJJ/blob/main/ex14_%EB%84%A4%EC%9D%B4%EB%B2%84%EC%98%81%ED%99%94%EB%8C%93%EA%B8%80%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Mining
  - 자연어 처리 (NLP : Natural Language Processing) : 사람이 사용하는 말을 분석하여 컴퓨터가 처리할 수 있도록 하는 작업

  - 분야 : 요약, 번역, 감성분석, 카테고리분류, 스팸 메일 분석, 질의응답, 챗봇, 검색엔진 등

  - ChatGPT + Dalle3 → 텍스트를 입력하면 이미지를 생성 (저작권, 지식재산권 이슈 방지)

# 자연어 처리 과정
  - 코퍼스 (corpus) : 처리할 문장
- (1) 토큰화 : 코퍼스를 분리하는 작업 (작은 문장, 단어, 문자, 자소 등)
- (2) 정제(Cleaning) : 오류 수정 (오타 등 수정)
- (3) 정규화 : 토큰(단어) 통합
  - 어간추출 (Stemming) : 단어의 원형으로 통합
    - 만들다, 만들었어요, 만들까요 → 만듦
  - 표제어추출 (Lemmatization) : 대표 단어로 통합
    - 죽다, 돌아가셨다, 갔다, 승천하셨다 → 죽음
  
- (4) 전처리
  - 정규표현식 사용하여 전처리 ( 한글만 추출, 영어만 추출)
  - 불용어 (stopword) 처리 : 학습에 사용하지 않는 단어를 제거하는 작업
  - 내부단어분리 : 여러개의 단어로 만들어진 단어를 토큰화하는 작업
    - 스마트인재개발원 → 스마트, 인재, 개발원

- (5) 인코딩 : 단어 → 숫자
  - 정수인코딩 : 단어 → 정수
    - 빈도수 분석
    - 빈도수 순으로 정렬
    - 정렬된 순으로 인덱스를 부여 (1부터 시작, 0은 없음을 의미하기 때문)

  - word vector : 정수 → 실수
    - TF-IDF

- (6) padding 작업 : 입력 데이터를 같은 길이로 만들어 주는 작업
  - 짧은 데이터 : 부족한 길이만큼 앞쪽에 빈 공백을 채움
  - 긴 데이터 : 앞 부터 기준 길이만큼만 잘라냄

- (7) Embedding 층에 입력
  - 텍스트 데이터로부터 특성을 추출
  - 단어를 단어 간 거리와 방향(유사성)을 벡터 공간에 표현
  - 전체 단어수, 차원수, 입력 데이터의 길이

- (8) 시계열 분석 (RNN)
  - 시간에 따른 단어의 배치의 특성을 분석

- (9) Dense로 분류분석

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 한글이 토큰화가 어려운 이유
  - 띄어쓰기가 잘 지켜지지 않아서 빈 공백으로 분리하기가 어려움
  - 조사, 부사, 형용사 등이 존재

  - 토큰화 방법 - 형태소 분리 (명사, 부사, 형용사 등으로 분리) - 형태소 분리 라이브러리 사용 (Kkma, Okt, Hannaum, Mecab 등)

# 네이버 영화 댓글 감성 분석

- 총 20만개의 데이터로 구성
- 긍정 1, 부정 0으로 라벨링

In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/사물지능A

/content/drive/MyDrive/Colab Notebooks/사물지능A


In [3]:
# 한글 자연어 처리 라이브러리 설치
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 41.6 MB/s eta 0:00:00


In [4]:
import pandas as pd

data = pd.read_table("./data/ratings.txt")

print(len(data))

200000


In [5]:
# 데이터 확인
data[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


- 라벨의 클래스별 데이터 수 확인

In [6]:
data["label"].value_counts()

1    100000
0    100000
Name: label, dtype: int64

- 결측치 확인
  - 댓글 내용이 없는 데이터

In [7]:
data.isnull().sum()

id          0
document    8
label       0
dtype: int64

- 결측치가 있는 데이터 확인

In [8]:
data.loc[data["document"].isnull()]

,id,document,label
46471,6369843,NaN,1
60735,511097,NaN,1
77665,2172111,NaN,1
84098,402110,NaN,1
127017,5942978,NaN,0
172375,5026896,NaN,0
173526,1034280,NaN,0
197279,1034283,NaN,0


- 결측치 삭제

In [9]:
data = data.dropna(how="any") # how="any" : 하나의 컬럼이라도 결측치가 있다면 삭제

data.isnull().sum()

id          0
document    0
label       0
dtype: int64

- 댓글에서 한글과 빈 공백만 추출 (정규식)

In [10]:
# ^ : 부정 (아닌 것)
# ㄱ-ㅎ : 자음 (ex. ㅋㅋㅋ)
# ㅏ-ㅣ : 모음 (ex. ㅠㅠ)
# 가-힣 : 한글
#   : 빈공백
# 0-9 : 숫자
# A-Z, a-z : 영문
data["document"] = data["document"].str.replace("[^가-힣 ]","")

data[:5]

<ipython-input-10-c2619794949e>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  data["document"] = data["document"].str.replace("[^가-힣 ]","")


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1


In [11]:
# 결측치 확인
data.isnull().sum() # 추출 후의 데이터는 체크가 제대로 안됨!

id          0
document    0
label       0
dtype: int64

In [12]:
# 전처리(빈 공백 제거) 후에 생긴 빈 데이터 확인
data.loc[data["document"].str.strip() == ""] # 빈 공백 제거

,id,document,label
111,6906147,,1
190,7059515,,1
203,10216462,,1
608,8085167,,1
661,3579401,,1
...,...,...,...
199606,7922190,,0
199670,7238007,,0
199683,2205229,,0
199848,7727438,,0


In [13]:
# 빈 공백이 아닌 데이터만 다시 저장
data = data.loc[data["document"].str.strip() != ""]

data.loc[data["document"].str.strip() == ""]

,id,document,label


- 토큰화(형태소 분리), 불용어처리, 어간추출

In [15]:
import konlpy
from konlpy.tag import Okt
from tqdm import tqdm

okt = Okt()

data2 = []

stopword = ["의", "과", "이", "은", "는", "들", "좀", "걍", "도", "을", "를", "으로", "로", "자", "예", "와", "한", "하다", "하", "호", "꽤", "까지", "에서"]
for sen in tqdm(data["document"]) :
  temp = [] # 중간 결과 저장
  # 형태소 분리, 어간추출 옵션
  temp = okt.morphs(sen, stem=True)

  # 불용어 처리
  # 토큰화된 temp를 하나씩 읽어서 word에 저장하고
  # 저장된 word가 stopword에 없는 거라면(불용어가 아니라면)
  # word를 temp 리스트에 추가
  temp = [word for word in temp if word not in stopword]

  ### 동일한 코드
  # tempX = []
  # for word in temp:
  #   if word not in stopword :
  #     tempX.append(word)

  data2.append(temp=)

100%|██████████| 197815/197815 [12:52<00:00, 255.93it/s]


- 토큰화된 내용을 파일로 저장

In [16]:
import pickle # 데이터 타입을 보존하면서 저장

file_path = "./data/data2.txt"

with open(file_path, "wb") as f :
  pickle.dump(data2, f)

- 저장된 파일 불러오기

In [17]:
data3 = []

with open(file_path, "rb") as f :
  data3 = pickle.load(f)

In [18]:
data3[:5]

[['어리다', '때', '보고', '지금', '다시', '보다', '재밌다'],
 ['디자인',
  '배우다',
  '학생',
  '외국',
  '디자이너',
  '그',
  '일군',
  '전통',
  '통해',
  '발전',
  '문화',
  '산업',
  '부럽다',
  '사실',
  '우리나라',
  '에서도',
  '그',
  '어렵다',
  '시절',
  '에',
  '끝',
  '열정',
  '지키다',
  '노라노',
  '같다',
  '전통',
  '있다',
  '저',
  '같다',
  '사람',
  '꿈',
  '꾸다',
  '이루다',
  '나가다',
  '수',
  '있다',
  '것',
  '에',
  '감사하다'],
 ['폴리스스토리', '시리즈', '부터', '뉴', '버리다', '하나', '없다', '최고'],
 ['오다',
  '연기',
  '가',
  '진짜',
  '개',
  '쩔다',
  '지루하다',
  '생각',
  '몰입',
  '보다',
  '그렇다',
  '이렇다',
  '진짜',
  '영화',
  '지'],
 ['안개', '자욱하다', '밤하늘', '에', '뜨다', '있다', '초승달', '같다', '영화']]

- 인코딩 : 문자 → 숫자

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 사용할 최대 단어수
max_feature = 35000

# 인코딩
tokenizer = Tokenizer(num_words=max_feature)
# 인코딩하기 위해서 데이터 분석 (빈도수 분석)
tokenizer.fit_on_texts(data2)

# 정렬 → 단어 대신에 인덱스를 부여
data2 = tokenizer.texts_to_sequences(data2)

In [21]:
data2[:5]

[[169, 46, 40, 99, 77, 2, 12],
 [3596,
  466,
  1447,
  1352,
  7104,
  28,
  28002,
  4635,
  987,
  1322,
  1141,
  5866,
  1723,
  392,
  372,
  741,
  28,
  575,
  572,
  3,
  106,
  1353,
  1114,
  28003,
  17,
  4635,
  7,
  82,
  17,
  39,
  607,
  1432,
  1403,
  465,
  66,
  7,
  25,
  3,
  645],
 [6893, 250, 135, 4723, 129, 78, 5, 26],
 [84, 24, 4, 16, 97, 1005, 63, 32, 286, 2, 70, 41, 16, 1, 72],
 [5867, 21951, 13436, 3, 683, 7, 21952, 17, 1]]

- padding : 데이터 길이를 동일하게 만들어 주는 작업
  - 길이 짧으면 앞부분에 빈 공백을 채운다.

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 길이
max_word = 30

data2 = pad_sequences(data2, maxlen=max_word)

In [23]:
data2[:3]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,   169,    46,    40,    99,
           77,     2,    12],
       [ 1322,  1141,  5866,  1723,   392,   372,   741,    28,   575,
          572,     3,   106,  1353,  1114, 28003,    17,  4635,     7,
           82,    17,    39,   607,  1432,  1403,   465,    66,     7,
           25,     3,   645],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,  6893,   250,   135,  4723,   129,
           78,     5,    26]], dtype=int32)

In [29]:
label = data["label"]

- 훈련데이터와 검증데이터 분리

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(data2, label, random_state=1)

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((148361, 30), (49454, 30), (148361,), (49454,))

- 신경망 설계

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()

# 텍스트 데이터에서 특성을 뽑는 전처리기 역할
#   → 인코딩된 데이터를 다차원 공간에 벡터로 맵핑 (실수로 변경)
# 최대단어 수, 차원 수, 데이터 길이
# 차원 수 : 하나의 단어가 연관을 가지는 단어의 수
model.add(Embedding(max_feature, 100, input_length=max_word))

model.add(LSTM(64))

model.add(Dense(1, activation="sigmoid"))

In [34]:
model.compile(loss="binary_crossentropy",
              optimizer = "adam",
              metrics=["accuracy"])

In [37]:
ms = ModelCheckpoint("./data/naver_{epoch:02d}_{accuracy:.3f}_{val_accuracy:.3f}.hdf5",
                     monitor="val_accuracy",
                     save_best_only=True)
es = EarlyStopping(monitor="val_accuracy",
                   patience=5)

h = model.fit(X_train, y_train, epochs=5, batch_size=50, validation_data=(X_valid, y_valid), callbacks=[ms, es])

Epoch 1/5
2968/2968 [==============================] - ETA: 0s - loss: 0.2831 - accuracy: 0.8817

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2968/2968 [==============================] - 242s 82ms/step - loss: 0.2831 - accuracy: 0.8817 - val_loss: 0.3375 - val_accuracy: 0.8555
Epoch 2/5
2968/2968 [==============================] - 241s 81ms/step - loss: 0.2314 - accuracy: 0.9055 - val_loss: 0.3572 - val_accuracy: 0.8537
Epoch 3/5
2968/2968 [==============================] - 238s 80ms/step - loss: 0.1866 - accuracy: 0.9254 - val_loss: 0.4116 - val_accuracy: 0.8480
Epoch 4/5
2968/2968 [==============================] - 277s 93ms/step - loss: 0.1491 - accuracy: 0.9415 - val_loss: 0.4489 - val_accuracy: 0.8456
Epoch 5/5
2968/2968 [==============================] - 252s 85ms/step - loss: 0.1204 - accuracy: 0.9532 - val_loss: 0.5673 - val_accuracy: 0.8398


- 베스트 모델을 로드

In [38]:
from tensorflow.keras.models import load_model

best_model = load_model("./data/naver_01_0.882_0.856.hdf5")

- 예측

In [39]:
import numpy as np

pred = model.predict(X_valid)


1546/1546 [==============================] - 15s 9ms/step


In [41]:
print(pred[444])
print(y_valid.iloc[444])

[0.99243224]
1
